In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns

import gc


In [6]:
#if possibble_overflow= 0 then there is possibility to reduce all column types from int64 to int32
#without any information loss
#dropping id because its the only feature that exceed int16, so it will stay int32 anyway
def is_validate_for_reduction_uint16(df):
    df = df.drop(columns='id')
    min_uint16=0
    max_uint16=65535
    columns_to_reduce = []
    for col in df.columns:
        if not (df[col].min() < min_uint16 or df[col].max() > max_uint16):
            columns_to_reduce.append(col)
    return columns_to_reduce

def is_validate_for_reduction_uint8(df):
    df = df.drop(columns='id')
    min_uint16=0
    max_uint16=255
    columns_to_reduce = []
    for col in df.columns:
        if not (df[col].min() < min_uint16 or df[col].max() > max_uint16):
            columns_to_reduce.append(col)
    return columns_to_reduce

In [3]:
df = pd.DataFrame()
for num in range(0, 12):
    tmp_df = pd.read_parquet('data/data/train_data' + '/train_data_' + str(num) + '.pq')
    for column in is_validate_for_reduction_uint16(tmp_df):
        tmp_df[column] = tmp_df[column].astype('uint16')
    for column in is_validate_for_reduction_uint8(tmp_df):
        tmp_df[column] = tmp_df[column].astype('uint8')
    tmp_df['id'] = tmp_df['id'].astype('uint32')
    df = pd.concat([df, tmp_df])
    print(f'dataframe number {num} sucesfully concatenated.')
gc.collect()

dataframe number 0 sucesfully concatenated.
dataframe number 1 sucesfully concatenated.
dataframe number 2 sucesfully concatenated.
dataframe number 3 sucesfully concatenated.
dataframe number 4 sucesfully concatenated.
dataframe number 5 sucesfully concatenated.
dataframe number 6 sucesfully concatenated.
dataframe number 7 sucesfully concatenated.
dataframe number 8 sucesfully concatenated.
dataframe number 9 sucesfully concatenated.
dataframe number 10 sucesfully concatenated.
dataframe number 11 sucesfully concatenated.


0

In [3]:
target = pd.read_csv('data/target.csv')

id	Идентификатор заявки. Заявки пронумерованы так, что большему номеру соответствует более поздняя дата заявки.
rn	Порядковый номер кредитного продукта в кредитной истории. Большему номеру соответствует продукт с более поздней датой открытия.
pre_since_opened	Дней с даты открытия кредита до даты сбора данных (бинаризовано*)
pre_since_confirmed	Дней с даты подтверждения информации по кредиту до даты сбора данных (бинаризовано*)
pre_pterm	Плановое количество дней с даты открытия кредита до даты закрытия (бинаризовано*)
pre_fterm	Фактическое количество дней с даты открытия кредита до даты закрытия (бинаризовано*)
pre_till_pclose	Плановое количество дней с даты сбора данных до даты закрытия кредита (бинаризовано*)
pre_till_fclose	Фактическое количество дней с даты сбора данных до даты закрытия кредита (бинаризовано*)
pre_loans_credit_limit	Кредитный лимит (бинаризовано*)
pre_loans_next_pay_summ	Сумма следующего платежа по кредиту (бинаризовано*)
pre_loans_outstanding	Оставшаяся невыплаченная сумма кредита (бинаризовано*)
pre_loans_total_overdue	Текущая просроченная задолженность (бинаризовано*)
pre_loans_max_overdue_sum	Максимальная просроченная задолженность (бинаризовано*)
pre_loans_credit_cost_rate	Полная стоимость кредита (бинаризовано*)
pre_loans5	Число просрочек до 5 дней (бинаризовано*)
pre_loans530	Число просрочек от 5 до 30 дней (бинаризовано*)
pre_loans3060	Число просрочек от 30 до 60 дней (бинаризовано*)
pre_loans6090	Число просрочек от 60 до 90 дней (бинаризовано*)
pre_loans90	Число просрочек более, чем на 90 дней (бинаризовано*)
is_zero_loans_5	Флаг: нет просрочек до 5 дней
is_zero_loans_530	Флаг: нет просрочек от 5 до 30 дней
is_zero_loans_3060	Флаг: нет просрочек от 30 до 60 дней
is_zero_loans_6090	Флаг: нет просрочек от 60 до 90 дней
is_zero_loans90	Флаг: нет просрочек более, чем на 90 дней
pre_util	Отношение оставшейся невыплаченной суммы кредита к кредитному лимиту (бинаризовано*)
pre_over2limit	Отношение текущей просроченной задолженности к кредитному лимиту (бинаризовано*)
pre_maxover2limit	Отношенение максимальной просроченной задолженности к кредитному лимиту (бинаризовано*)
is_zero_util	Флаг: отношение оставшейся невыплаченной суммы кредита к кредитному лимиту равняется 0
is_zero_over2limit	Флаг: отношение текущей просроченной задолженности к кредитному лимиту равняется 0
is_zero_maxover2limit	Флаг: отношение максимальной просроченной задолженности к кредитному лимиту равняется 0
enc_paym_{0..N}	Статусы ежемесячных платежей за последние N месяцев (закодировано**)
enc_loans_account_holder_type	Тип отношения к кредиту (закодировано**)
enc_loans_credit_status	Статус кредита (закодировано**)
enc_loans_account_cur	Валюта кредита (закодировано**)
enc_loans_credit_type	Тип кредита (закодировано**)
pclose_flag	Флаг: плановое количество дней с даты открытия кредита до даты закрытия не определено 
fclose_flag	Флаг: фактическое количество дней с даты открытия кредита до даты закрытия не определено 
    
* область значений поля разбивается на N непересекающихся промежутков, каждому промежутку случайным образом ставится в соответствие уникальный номер от 0 до N-1, значение поля заменяется номером промежутка, которому оно принадлежит  	
** каждому уникальному значению поля случайным образом ставится в соответствие уникальный номер от 0 до K, значение поля заменяется номером этого значения	


In [5]:
df.columns

Index(['id', 'rn', 'pre_since_opened', 'pre_since_confirmed', 'pre_pterm',
       'pre_fterm', 'pre_till_pclose', 'pre_till_fclose',
       'pre_loans_credit_limit', 'pre_loans_next_pay_summ',
       'pre_loans_outstanding', 'pre_loans_total_overdue',
       'pre_loans_max_overdue_sum', 'pre_loans_credit_cost_rate', 'pre_loans5',
       'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90',
       'is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060',
       'is_zero_loans6090', 'is_zero_loans90', 'pre_util', 'pre_over2limit',
       'pre_maxover2limit', 'is_zero_util', 'is_zero_over2limit',
       'is_zero_maxover2limit', 'enc_paym_0', 'enc_paym_1', 'enc_paym_2',
       'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7',
       'enc_paym_8', 'enc_paym_9', 'enc_paym_10', 'enc_paym_11', 'enc_paym_12',
       'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16',
       'enc_paym_17', 'enc_paym_18', 'enc_paym_19', 'enc_paym_20',
       'enc_paym_21', 

In [6]:
df.head(15)


,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,...,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,...,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,...,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,...,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,...,3,3,3,4,1,3,4,1,0,0
5,0,6,5,0,11,8,12,11,4,2,...,3,3,3,4,1,2,3,1,0,1
6,0,7,3,9,1,2,12,14,15,5,...,3,3,3,4,1,3,4,1,0,0
7,0,8,2,9,2,3,12,14,15,5,...,3,3,3,4,1,3,4,1,0,0
8,0,9,1,9,11,13,14,8,2,5,...,3,3,3,4,1,2,4,1,0,0
9,0,10,7,9,2,10,8,8,16,4,...,3,3,3,4,1,2,4,1,0,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26162717 entries, 0 to 2450629
Data columns (total 61 columns):
 #   Column                         Dtype 
---  ------                         ----- 
 0   id                             uint32
 1   rn                             uint8 
 2   pre_since_opened               uint8 
 3   pre_since_confirmed            uint8 
 4   pre_pterm                      uint8 
 5   pre_fterm                      uint8 
 6   pre_till_pclose                uint8 
 7   pre_till_fclose                uint8 
 8   pre_loans_credit_limit         uint8 
 9   pre_loans_next_pay_summ        uint8 
 10  pre_loans_outstanding          uint8 
 11  pre_loans_total_overdue        uint8 
 12  pre_loans_max_overdue_sum      uint8 
 13  pre_loans_credit_cost_rate     uint8 
 14  pre_loans5                     uint8 
 15  pre_loans530                   uint8 
 16  pre_loans3060                  uint8 
 17  pre_loans6090                  uint8 
 18  pre_loans90               

In [8]:
def countAggregate(df, iternum):
    feature_columns = list(df.columns.values)
    feature_columns.remove("id")
    feature_columns.remove("rn")
    
    dummies = pd.get_dummies(df[feature_columns], columns=feature_columns)
    dummy_features = dummies.columns.values

    ohe_features = pd.concat([df, dummies], axis=1)
    ohe_features = ohe_features.drop(columns=feature_columns)

    features = ohe_features.groupby("id")[dummy_features].sum().reset_index(drop=False)

    count_df = ohe_features.groupby("id").size().reset_index(name='loan_counts')
    features = features.merge(count_df, on="id")
    
    for column in is_validate_for_reduction_uint8(features):
        features[column] = features[column].astype('uint8')

    
    
    
    features.to_pickle(f"data_all_{iternum}.pkl")


In [ ]:

df = countAggregate(df,1)


df.info()

In [ ]:
df = target.merge(df, on="id")
df = df.drop(columns=["id"])

In [9]:
for num in range(0, 12):
    tmp_df = pd.read_parquet('data/data/train_data' + '/train_data_' + str(num) + '.pq')
    for column in is_validate_for_reduction_uint16(tmp_df):
        tmp_df[column] = tmp_df[column].astype('uint16')
    for column in is_validate_for_reduction_uint8(tmp_df):
        tmp_df[column] = tmp_df[column].astype('uint8')
    tmp_df['id'] = tmp_df['id'].astype('uint32')
    
    countAggregate(tmp_df,num)

    print(f'dataframe number {num} sucesfully written.')

dataframe number 0 sucesfully written.
dataframe number 1 sucesfully written.
dataframe number 2 sucesfully written.
dataframe number 3 sucesfully written.
dataframe number 4 sucesfully written.
dataframe number 5 sucesfully written.
dataframe number 6 sucesfully written.
dataframe number 7 sucesfully written.
dataframe number 8 sucesfully written.
dataframe number 9 sucesfully written.
dataframe number 10 sucesfully written.
dataframe number 11 sucesfully written.


In [10]:
df = pd.DataFrame()
for num in range(0, 12):
    tmp_df = pd.read_pickle(f"data_all_{num}.pkl")
    df = pd.concat([df, tmp_df])

target = pd.read_csv('data/target.csv')
df = target.merge(df, on="id")
df["flag"] = df["flag"].astype('category')
df = df.drop(columns=["id"])
df.fillna(np.uint8(0), inplace=True)
float64_col = df.select_dtypes(['float64']).columns
df[float64_col] = df[float64_col].astype('uint8')
df.to_pickle(f"data_all.pkl")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Columns: 421 entries, flag to pre_loans530_17
dtypes: category(1), uint8(420)
memory usage: 1.2 GB
